# Course Project

## Dataset

You will examine the ProPublica COMPAS dataset, which consists of all criminal defendants who were subject to COMPAS screening in Broward County, Florida, during 2013 and 2014. For each defendant, various information fields (‘features’) were also gathered by ProPublica. Broadly, these fields are related to the defendant’s demographic information (e.g., gender and race), criminal history (e.g., the number of prior offenses) and administrative information about the case (e.g., the case number, arrest date, risk of recidivism predicted by the COMPAS tool). Finally, the dataset also contains information about whether the defendant did actually recidivate or not.

The COMPAS score uses answers to 137 questions to assign a risk score to defendants -- essentially a probability of re-arrest. The actual output is two-fold: a risk rating of 1-10 and a "low", "medium", or "high" risk label.

Link to dataset: https://github.com/propublica/compas-analysis

The file we will analyze is: compas-scores-two-years.csv

Link to the ProPublica article:

https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing


## Project goal

The project has three parts: 

- The COMPAS scores have been shown to have biases against certain racial groups. Analyze the dataset to highlight these biases.  

- Based on the features in the COMPAS dataset, train classifiers to predict who will re-offend (hint: no need to use all features, just the ones you find relevant).  Study if your classifiers are more or less fair than the COMPAS classifier. 

- Build a fair classifier (last lecture will cover fair classification techniques). Is excluding the race from the feature set enough?


## Today

Explore the dataset and do some initial statistics. 

In [ ]:
!pip install tensorflow

## Download the data

We first need to load the data from the ProPublica repo:
https://github.com/propublica/compas-analysis


In [ ]:
import urllib
import os,sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import feature_extraction
from sklearn import preprocessing
from sklearn.metrics import accuracy_score,confusion_matrix

from random import seed, shuffle
#from __future__ import division
#from collections import defaultdict
#import utils as ut

SEED = 1234
seed(SEED)
np.random.seed(SEED)
pd.set_option('display.max_columns', None)
def check_data_file(fname):
    files = os.listdir(".") # get the current directory listing
    print("Looking for file '%s' in the current directory...",fname)

    if fname not in files:
        print("'%s' not found! Downloading from GitHub...",fname)
        addr = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
        response = urllib.request.urlopen(addr)
        data = response.read()
        fileOut = open(fname, "wb")
        fileOut.write(data)
        fileOut.close()
        print("'%s' download and saved locally..",fname)
    else:
        print("File found in current directory..")
    
COMPAS_INPUT_FILE = "compas-scores-two-years.csv"
check_data_file(COMPAS_INPUT_FILE)  

## Load data and clean it up

__Load the data__

hint: data is in csv format; pandas is a python library that can read csv files

you can choose to represent your data either as a DataFrame or as a dictionary

- The dataset contains data on how many convicts? 

- What are the features the dataset contains?

hint pandas: check pandas functions shape, column, head

hint dictionary: check keys() function

__Cleanup the data__

- Are there missing values (NaN)? are there outliers?  

hint pandas: check isnull function in pandas

hint dictionary: implement a for and check if the variable is None

- Does ProPublica mentions how to clean the data?  

__What is the effect of the following function?__

df = pd.read_csv(COMPAS_INPUT_FILE)

print(df.shape)

df = df.dropna(subset=["days_b_screening_arrest"]) # dropping missing vals

df = df[
    (df.days_b_screening_arrest <= 30) &  
    (df.days_b_screening_arrest >= -30) &  
    (df.is_recid != -1) &
    (df.c_charge_degree != 'O') &
    (df.score_text != 'N/A')
]

df.reset_index(inplace=True, drop=True) # renumber the rows from 0 again

In [ ]:
df = pd.read_csv(COMPAS_INPUT_FILE)
df.head()

## Basic analysis of demographics

- **What are the different races present in the dataset?** 

- **What is the number of people by age category?**

- **What is the number of people by race?**

- **What is the number of people by COMPAS score (decile_score)?**

- **What is the number of people by COMPAS risk category (score_text)?**

In [ ]:
df.groupby("age_cat").size()
df.groupby("decile_score").size()
df.groupby("score_text").size()
df["race"].unique().tolist()
df.groupby("race").size()

## Basic investigations of gender and race bias in COMPAS scores

*decile_score -- is the score given by the COMPAS algorithm that estimates the risk to re-offend.*

*score_text -- is the level of risk: Low, Medium, High*

*two_years_recid -- is the ground truth data on whether the offender recidivated or not* 

- **What is the mean COMPAS score (decile_score) per race and gender?** 

In [ ]:
print(df.groupby(["race","sex"]).mean()["decile_score"])

**Comment**: Since there are only 11 Native American people in this data set we think we should disregard them. It is the same for the Asian group, we think there are simply not enough Asian people in order to have an unbised estimator for Asian people. For this reason, we will remove the categories "Native American" and "Asian" and place these samples in the category "Other". For all races we can clearly see there is more data for male than there are for female so we will have to deal with this unbalance in some way.

In [ ]:
df.loc[(df.race == 'Asian'),'race']='Other'
df.loc[(df.race == 'Native American'),'race']='Other'

- **What is the distribution (histogram) of decile_score per race and gender?**

In [ ]:
df.hist(column='decile_score', by=["race","sex"],figsize = (11,12));
plt.show()

**Comment**: As we can see there seems to be a bias towards African American people since the histogram distribution on this race is a lot more evenly distributed when compared to the other races histograms (we can observe a higher percentage of higher scores in the African American race). For the other races we can clearly see that the  histogram is a lot more squewed to lower values. This applies both for men and women.

*The two_year_recid field records whether or not each person was re-arrested for a violent offense within two years, which is what COMPAS is trying to predict.*

- **How many people were re-arrested?** 

- **Compute the recidivism (i.e., people that got re-arrested) rates by race and gender**

In [ ]:
df.groupby("two_year_recid").size()

In [ ]:
races = df["race"].unique().tolist()

for race in races:
    df.loc[df['race'] == race].groupby(["race","two_year_recid"]).size().plot(kind = "pie", subplots=True , title = "Re-arrested rate by race",autopct='%1.0f%%',label ="", figsize=(3,3)) 
    plt.show()
    
df.loc[df['sex'] == "Male"].groupby(["sex","two_year_recid"]).size().plot(kind = "pie", subplots=True , title = "Re-arrested rate by race",autopct='%1.0f%%',label ="", figsize=(3,3)) 
plt.show()

df.loc[df['sex'] == "Female"].groupby(["sex","two_year_recid"]).size().plot(kind = "pie", subplots=True , title = "Re-arrested rate by race",autopct='%1.0f%%',label ="", figsize=(3,3)) 
plt.show()
    
    
#try bar chart later

In [ ]:
races_names = list(df["race"].unique().tolist())
races_size = list(df.groupby("race").size())

plt.bar(races_names, races_size)
plt.show()

In [ ]:
dfr = df[["decile_score","two_year_recid","sex","race"]]

def is_recid(row):
    if(row["decile_score"] >= 5):
        return 1
    else:
        return 0
    
dfr['pred_is_recid'] = dfr.apply (lambda row: is_recid(row), axis=1)
dfr.head()


- **What is the accuracy of the COMPAS scores to predict recidivism**

- **Is the accuracy higher/lower if we look at particular races/genders?**

- **What about false positives and false negatives?**

In [ ]:
accuracy_score(dfr["two_year_recid"], dfr["pred_is_recid"])

In [ ]:
races = df["race"].unique().tolist()
genders = ["Male","Female"]
for race in races:
    aux_df = dfr.loc[dfr['race'] == race]
    acc = accuracy_score(aux_df["two_year_recid"], aux_df["pred_is_recid"])
    print("Acc for {race} is {acc}".format(race = race, acc = acc))

for sex in genders:
    aux_df = dfr.loc[dfr['sex'] == sex]
    acc = accuracy_score(aux_df["two_year_recid"], aux_df["pred_is_recid"])
    print("Acc for {sex} is {acc}".format(sex = sex, acc = acc))

In [ ]:
def makeCM (dfr,label):
    cf_matrix = confusion_matrix(dfr["two_year_recid"], dfr["pred_is_recid"])

    group_names = ['True Neg','False Pos','False Neg','True Pos']

    group_counts = ["{0:0.0f}".format(value) for value in
                    cf_matrix.flatten()]

    group_percentages = ["{0:.2%}".format(value) for value in
                         cf_matrix.flatten()/np.sum(cf_matrix)]

    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
              zip(group_names,group_counts,group_percentages)]

    labels = np.asarray(labels).reshape(2,2)

    ax = sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')

    ax.set_title('Confusion Matrix for ' + label);
    ax.set_xlabel('\nPredicted Values')
    ax.set_ylabel('Actual Values ');

    ## Ticket labels - List must be in alphabetical order
    ax.xaxis.set_ticklabels(['False','True'])
    ax.yaxis.set_ticklabels(['False','True'])

    ## Display the visualization of the Confusion Matrix.
    plt.show()

In [ ]:
races = df["race"].unique().tolist()
genders = ["Male","Female"]
for race in races:
    aux_df = dfr.loc[dfr['race'] == race]
    makeCM(aux_df,race)  

for sex in genders:
    aux_df = dfr.loc[dfr['sex'] == sex]
    makeCM(aux_df,sex)

**Comment**: From the confusion matrices we can see that the false positive rate is highest among african americans indicating that there's a bias against them. The opposite can be found for the other races. The False Negatives is then lower for African American than for other races meaning that the model does a bad job at identifying people that will recomit a crime if they are not african american.

# Algorithms

In [ ]:
dfa = df[["sex","age_cat","race","priors_count","two_year_recid","juv_misd_count","juv_fel_count","juv_other_count","c_charge_degree"]]

In [ ]:
dfa.loc[(df.race == 'Asian'),'race']='Other'
dfa.loc[(df.race == 'Native American'),'race']='Other'
dfa

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
column_trans = make_column_transformer(
    (OneHotEncoder(), ['sex', 'age_cat',"c_charge_degree","race"]),
    remainder='passthrough')
d = column_trans.fit_transform(dfa)




In [ ]:
labels = column_trans.get_feature_names_out()

def cleanLabels(label):
    if("onehotencoder__" in label):
        return label.removeprefix("onehotencoder__")
    else:
        return label.removeprefix("remainder__")
cleanedLabels = map(cleanLabels,labels)
listedLabels = list(map(cleanLabels,labels))

In [ ]:
dfa_num = pd.DataFrame(data=d,columns=cleanedLabels)
dfa_num

In [ ]:
dfa_num.columns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
Y = dfa_num["two_year_recid"]
X = dfa_num.drop(columns="two_year_recid")

X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.30, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.predict(X_test)
clf.score(X_test,y_test)
#cross validation for reg

In [ ]:
scores = []
for c in np.arange (0.01, 1, 0.05):
    clf = LogisticRegression(random_state=0,C = c)
    cv = cross_validate(clf, X_train, y_train, cv=10,scoring='f1_macro')
    score = np.mean(cv["test_score"])
    scores.append([score,c])
reg = max(scores)
print(reg)

In [ ]:
clf = LogisticRegression(random_state=0,C = reg[1] ).fit(X_train, y_train)
clf.predict(X_test)
clf.score(X_test,y_test)

In [ ]:
importance = clf.coef_[0]
# summarize feature importance
j = 0
for i,v in enumerate(importance):
    print('Feature: %0s, Score: %.5f' % (listedLabels[j],v))
    j += 1
# plot feature importance
print(len(listedLabels))
print(len(importance))
plt.figure(figsize=(10, 5), dpi=80)
plt.bar(listedLabels[:-1], importance)
plt.rcParams['font.size'] = '25'
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Re-add asians and native americans
#Testing from which number to split from
#we can drop some cols

In [ ]:
from sklearn.svm import SVC
svc = make_pipeline(StandardScaler(), SVC(gamma='auto'))
svc.fit(X_train, y_train)
svc.score(X_test,y_test)
pred = svc.predict(X_test)
print( "Accuracy " + str(svc.score(X_test,y_test)))
f1 = f1_score(y_test, pred, average='macro')
print("F1-Score " + str(f1))

In [ ]:
scores = []
for c in np.arange (0.01, 1, 0.05):
    svc = make_pipeline(StandardScaler(), SVC(gamma='auto', C = c))
    cv = cross_validate(svc, X_train, y_train, cv=10)
    score = np.mean(cv["test_score"])
    scores.append([score,c])
reg = max(scores)
print(reg)

In [ ]:
best_svc = make_pipeline(StandardScaler(), SVC(gamma='auto',C = reg[1]))
best_svc.fit(X_train, y_train)
pred = best_svc.predict(X_test)
print( "Accuracy " + str(best_svc.score(X_test,y_test)))
f1 = f1_score(y_test, pred, average='macro')
print("F1-Score " + str(f1))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
k_scores = []
for k in range(3,12,2):
    neigh = KNeighborsClassifier(n_neighbors=k)
    cv = cross_validate(neigh, X_train, y_train, cv=5)
    k_score = np.mean(cv["test_score"])
    k_scores.append([k_score,k])
k_reg = max(k_scores)
print(k_reg)

In [ ]:
best_neigh = KNeighborsClassifier(n_neighbors=k_reg[1])
best_neigh.fit(X_train, y_train)
pred = best_neigh.predict(X_test)
print( "Accuracy " + str(best_neigh.score(X_test,y_test)))
f1 = f1_score(y_test, pred, average='macro')
print("F1-Score " + str(f1))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
##code from https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)


In [ ]:
print(rf_random.best_params_)
rf_random.best_params_["min_samples_leaf"]

In [ ]:
best_rf = RandomForestClassifier(max_depth=rf_random.best_params_["max_depth"], random_state=0, n_estimators=rf_random.best_params_["n_estimators"],min_samples_split=rf_random.best_params_["min_samples_split"],min_samples_leaf=rf_random.best_params_["min_samples_leaf"],max_features=rf_random.best_params_["max_features"],bootstrap=rf_random.best_params_["bootstrap"])
best_rf.fit(X_train, y_train)
pred = best_rf.predict(X_test)
print( "Accuracy " + str(best_rf.score(X_test,y_test)))
f1 = f1_score(y_test, pred, average='macro')
print("F1-Score " + str(f1))

In [ ]:
X.columns

In [ ]:
Index(['sex_Female', 'sex_Male', 'age_cat_25 - 45', 'age_cat_Greater than 45',
       'age_cat_Less than 25', 'c_charge_degree_F', 'c_charge_degree_M',
       'race_African-American', 'race_Caucasian', 'race_Hispanic',
       'race_Other', 'priors_count', 'two_year_recid', 'juv_misd_count',
       'juv_fel_count', 'juv_other_count'],
      dtype='object')

In [ ]:
#https://medium.com/luca-chuangs-bapm-notes/build-a-neural-network-in-python-binary-classification-49596d7dcabf
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping


model = Sequential()
model.add(Dense(64, input_shape=(X.shape[1],), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary() 

# compile the model
model.compile(optimizer='Adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

# early stopping callback
# This callback will stop the training when there is no improvement in  
# the validation loss for 10 consecutive epochs.  
es = EarlyStopping(monitor='val_accuracy', 
                                   mode='max', # don't minimize the accuracy!
                                   patience=10,
                                   restore_best_weights=True)

# now we just update our model fit call
history = model.fit(X_train,
                    y_train,
                    callbacks=[es],
                    epochs=500, # you can set this to a big number!
                    batch_size=10,
                    validation_data=(X_test,y_test),
                    shuffle=True,
                    verbose=1)



history_dict = history.history
# Learning curve(Loss)
# let's see the training and validation loss by epoch

# loss
loss_values = history_dict['loss'] # you can change this
val_loss_values = history_dict['val_loss'] # you can also change this

# range of X (no. of epochs)
epochs = range(1, len(loss_values) + 1) 

# plot
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'orange', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


# Learning curve(accuracy)
# let's see the training and validation accuracy by epoch

# accuracy
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# range of X (no. of epochs)
epochs = range(1, len(acc) + 1)

# plot
# "bo" is for "blue dot"
plt.plot(epochs, acc, 'bo', label='Training accuracy')
# orange is for "orange"
plt.plot(epochs, val_acc, 'orange', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# this is the max value - should correspond to
# the HIGHEST train accuracy
#np.max(val_acc)

score = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', score[1])


